In [ ]:
%pylab inline
import networkx as nx
import pandas as pd

In [ ]:
from utils import *

In [ ]:
import os

In [ ]:
import pickle as pkl

In [ ]:
DATASETS = ['contact-high-school', 'contact-primary-school']
SG = 'sg'
HASSE_TYPE = 'uniform'
WORK_FOLDER = './'

In [ ]:
P = 1.
N = 10
WALKLEN = 80
SEED = 0

In [ ]:
TUPLE_SIZE = 3 # 4

In [ ]:
embdim_list = [8, 16, 32, 64, 128, 256, 512, 1024]

### Save Reconstruction Scores

In [ ]:
for DATASET in DATASETS:

    hyperedges_path = WORK_FOLDER + 'processed-output/hyperedges/%s/'%(DATASET)

    positive_ex = np.load(hyperedges_path + '%s_pos_%s_%dstring.npz'%('reconstruction', 'all', TUPLE_SIZE),
                             allow_pickle=True)['arr_0']
    negative_ex = np.load(hyperedges_path + '%s_neg_%s_%dstring.npz'%('reconstruction', 'all', TUPLE_SIZE),
                             allow_pickle=True)['arr_0']
    negative_bounds = np.load(hyperedges_path + '%s_neg_%s_%dbounds.npz'%('reconstruction', 'all', TUPLE_SIZE),
                             allow_pickle=True)['arr_0']

    for k in range(TUPLE_SIZE-1):

        for EMBDIM in embdim_list:    
            PARAMS = '%s_%s_%s_%s' %\
                        ( 'dim'+str(EMBDIM), 'n'+str(N), 'p'+str(P), 'walklen'+str(WALKLEN))

            load_path = WORK_FOLDER + 'processed-output/embeddings/%s/%s/'%(DATASET, PARAMS)

            save_path = WORK_FOLDER + 'processed-output/figures/%s/%s/'%(DATASET, PARAMS)
            os.makedirs(save_path, exist_ok=True)

            if os.path.isdir(load_path):

                #Load Embeddings
                with open(load_path+'%d-s2vembs_%s_%s_maxorder%s.%s.pkl'%\
                            (k, SG, HASSE_TYPE, k, SEED), 'rb') as fh:
                    model_wv = pkl.load(fh)
                names = list(model_wv.keys())
                index = dict(zip(names, range(len(names))))
                model_wv = np.array([model_wv[n] for n in names])

                for nlink in range(TUPLE_SIZE+1):

                    for norder in [k]:

                        if norder>0 and nlink<TUPLE_SIZE:
                            continue

                        results_file = save_path+'dict-reconstruction-%dbounds-%dsimplex-max5000-%s-%s-simplexorder%d-maxorder%d.pkl'%\
                                (nlink, norder, SG, HASSE_TYPE, TUPLE_SIZE-1, k)
#                         if not os.path.isfile(results_file):
                        
                        dict_list = classification_score_from_x(positive_ex, 
                                                             negative_ex[negative_bounds==nlink], 
                                                             model_wv, index, norder)
                        pkl.dump(dict_list, open(results_file, 'wb'), protocol=pkl.HIGHEST_PROTOCOL)
            
                        print('DONE:', DATASET, TUPLE_SIZE, k, PARAMS, nlink, norder)

### Reconstruction Scores - Search 

In [ ]:
for DATASET in DATASETS:

    save_path = WORK_FOLDER + 'processed-output/figures/%s/'%(DATASET)

    for k in range(TUPLE_SIZE-1):
        for nlink in range(TUPLE_SIZE+1):

             for norder in [k]:

                if norder>0 and nlink<TUPLE_SIZE:
                    continue

                output_file =save_path + '/best-aucpr-reconstruction-%dbounds-%dsimplex-max5000-%s-%s-simplexorder%d-maxorder%d.pkl'%\
                            (nlink, norder, SG, HASSE_TYPE, TUPLE_SIZE-1, k)

#                 if not os.path.isfile(output_file):

                best_dim_list = []

                for EMBDIM in embdim_list:    
                    PARAMS = '%s_%s_%s_%s' %\
                    ( 'dim'+str(EMBDIM), 'n'+str(N), 'p'+str(P), 'walklen'+str(WALKLEN))

                    load_path = WORK_FOLDER + 'processed-output/figures/%s/%s/'%(DATASET, PARAMS)

                    results_file = load_path + 'dict-reconstruction-%dbounds-%dsimplex-max5000-%s-%s-simplexorder%d-maxorder%d.pkl'%\
                            (nlink, norder, SG, HASSE_TYPE, TUPLE_SIZE-1, k)
                    dict_list = pkl.load(open(results_file, 'rb'))
                    recon_scores = [d['auc_pr'] if d['y_test'] is not None else None for d in dict_list] 
                    if recon_scores[0] is not None:
                        best_dim_list.append((np.mean(recon_scores), EMBDIM))
                    else:
                        best_dim_list.append((None, EMBDIM))

                BEST = sorted(best_dim_list, reverse=True)[0][1]
                PARAMS = '%s_%s_%s_%s' %\
                    ( 'dim'+str(BEST), 'n'+str(N), 'p'+str(P), 'walklen'+str(WALKLEN))
                load_path = WORK_FOLDER + 'processed-output/figures/%s/%s/'%(DATASET, PARAMS)
 
                input_file = load_path + 'dict-reconstruction-%dbounds-%dsimplex-max5000-%s-%s-simplexorder%d-maxorder%d.pkl'%\
                            (nlink, norder, SG, HASSE_TYPE, TUPLE_SIZE-1, k)

                dict_list = pkl.load(open(input_file, 'rb'))
                pkl.dump(dict_list, open(output_file, 'wb'), protocol=pkl.HIGHEST_PROTOCOL)
                
                print('DONE:', DATASET, TUPLE_SIZE, k, PARAMS)

### Save Prediction Scores

In [ ]:
for DATASET in DATASETS:

    hyperedges_path = WORK_FOLDER + 'processed-output/hyperedges/%s/'%(DATASET)

    positive_ex = np.load(hyperedges_path + '%s_pos_%s_%dstring.npz'%('prediction', 'all', TUPLE_SIZE),
                             allow_pickle=True)['arr_0']
    positive_bounds = np.load(hyperedges_path + '%s_pos_%s_%dbounds.npz'%('prediction', 'all', TUPLE_SIZE),
                             allow_pickle=True)['arr_0']
    negative_ex = np.load(hyperedges_path + '%s_neg_%s_%dstring.npz'%('prediction', 'all', TUPLE_SIZE),
                             allow_pickle=True)['arr_0']
    negative_bounds = np.load(hyperedges_path + '%s_neg_%s_%dbounds.npz'%('prediction', 'all', TUPLE_SIZE),
                             allow_pickle=True)['arr_0']

    for k in range(TUPLE_SIZE-1):

        for EMBDIM in embdim_list:    
            PARAMS = '%s_%s_%s_%s' %\
                        ( 'dim'+str(EMBDIM), 'n'+str(N), 'p'+str(P), 'walklen'+str(WALKLEN))

            load_path = WORK_FOLDER + 'processed-output/embeddings/%s/%s/'%(DATASET, PARAMS)

            save_path = WORK_FOLDER + 'processed-output/figures/%s/%s/'%(DATASET, PARAMS)
            os.makedirs(save_path, exist_ok=True)

            if os.path.isdir(load_path):

                #Load Embeddings
                with open(load_path+'%d-s2vembs_%s_%s_maxorder%s.%s.pkl'%\
                            (k, SG, HASSE_TYPE, k, SEED), 'rb') as fh:
                    model_wv = pkl.load(fh)
                names = list(model_wv.keys())
                index = dict(zip(names, range(len(names))))
                model_wv = np.array([model_wv[n] for n in names])

                for nlink in range(TUPLE_SIZE+1):

                    for norder in [k]:

                        if norder>0 and nlink<TUPLE_SIZE:
                            continue

                        results_file = save_path+'dict-prediction-%dbounds-%dsimplex-max5000-%s-%s-simplexorder%d-maxorder%d.pkl'%\
                                (nlink, norder, SG, HASSE_TYPE, TUPLE_SIZE-1, k)
#                         if not os.path.isfile(results_file):
                        
                        dict_list = classification_score_from_x(positive_ex[positive_bounds==nlink], 
                                                             negative_ex[negative_bounds==nlink], 
                                                             model_wv, index, norder)
                        pkl.dump(dict_list, open(results_file, 'wb'), protocol=pkl.HIGHEST_PROTOCOL)
            
                        print('DONE:', DATASET, TUPLE_SIZE, k, PARAMS, nlink, norder)

### Prediction Scores - Search 

In [ ]:
for DATASET in DATASETS:
    
    save_path = WORK_FOLDER + 'processed-output/figures/%s/'%(DATASET)

    for k in range(TUPLE_SIZE-1):
        for nlink in range(TUPLE_SIZE+1):

             for norder in [k]:

                if norder>0 and nlink<TUPLE_SIZE:
                    continue

                output_file =save_path + '/best-aucpr-prediction-%dbounds-%dsimplex-max5000-%s-%s-simplexorder%d-maxorder%d.pkl'%\
                            (nlink, norder, SG, HASSE_TYPE, TUPLE_SIZE-1, k)

#                 if not os.path.isfile(output_file):

                best_dim_list = []

                for EMBDIM in embdim_list:    
                    PARAMS = '%s_%s_%s_%s' %\
                    ( 'dim'+str(EMBDIM), 'n'+str(N), 'p'+str(P), 'walklen'+str(WALKLEN))

                    load_path = WORK_FOLDER + 'processed-output/figures/%s/%s/'%(DATASET, PARAMS)

                    results_file = load_path + 'dict-prediction-%dbounds-%dsimplex-max5000-%s-%s-simplexorder%d-maxorder%d.pkl'%\
                            (nlink, norder, SG, HASSE_TYPE, TUPLE_SIZE-1, k)
                    dict_list = pkl.load(open(results_file, 'rb'))
                    pred_scores = [d['auc_pr'] if d['y_test'] is not None else None for d in dict_list] 
                    if pred_scores[0] is not None:
                        best_dim_list.append((np.mean(pred_scores), EMBDIM))
                    else:
                        best_dim_list.append((None, EMBDIM))

                BEST = sorted(best_dim_list, reverse=True)[0][1]
                PARAMS = '%s_%s_%s_%s' %\
                    ( 'dim'+str(BEST), 'n'+str(N), 'p'+str(P), 'walklen'+str(WALKLEN))
                load_path = WORK_FOLDER + 'processed-output/figures/%s/%s/'%(DATASET, PARAMS)

                
                input_file = load_path + 'dict-prediction-%dbounds-%dsimplex-max5000-%s-%s-simplexorder%d-maxorder%d.pkl'%\
                            (nlink, norder, SG, HASSE_TYPE, TUPLE_SIZE-1, k)

                dict_list = pkl.load(open(input_file, 'rb'))
                pkl.dump(dict_list, open(output_file, 'wb'), protocol=pkl.HIGHEST_PROTOCOL)
                
                print('DONE:', DATASET, TUPLE_SIZE, k, PARAMS)